# Análisis de redes mediante Análisis Nodal Modificado
<img src="./img/logo_UTN.svg" align="right" width="150" /> 

#### Por Mariano Llamedo Soria

### Resumen 
En este documento se presentan ...

* Funciones de análisis de cuadripolos: [smna](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/cuadripolos/index.html#pytc2.cuadripolos.smna)
* De presentación algebraica: [print_latex](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/general/index.html#pytc2.general.print_latex), [a_equal_b_latex_s](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/general/index.html#pytc2.general.a_equal_b_latex_s), [print_subtitle](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/general/index.html#pytc2.general.print_subtitle)
* De manipulación de sistemas lineales: [tf2sos_analog](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/sistemas_lineales/index.html#pytc2.sistemas_lineales.tf2sos_analog), [parametrize_sos](https://pytc2.readthedocs.io/en/latest/autoapi/pytc2/sistemas_lineales/index.html#pytc2.sistemas_lineales.parametrize_sos)


### Introducción

Documento en elaboración. Algunas referencias para este tema:

* Cap. 17 [Vlach, Jiri - Linear Circuit Theory_ Matrices in Computer Applications-Apple Academic Press (2014)](https://drive.google.com/file/d/1-H73vpb3ZST5bsALNvPFGv8a5E3l-R1R/view?usp=drive_link)
* [Erik Cheever. SCAM: Symbolic Circuit Analysis in MatLab
. Swarthmore University](https://lpsa.swarthmore.edu/Systems/Electrical/mna/MNA1.html)
* [Tony a.k.a *Tiburonboy*. Symbolic Modified Nodal Analysis using Python](https://tiburonboy.github.io/Symbolic-Modified-Nodal-Analysis-using-Python/)


### GIC bicuad

A partir de una red de referencia:

<img src="./img/GIC_bicuad2.png" alt="GIC Schaumann" width="800px" style="display: block; margin: 0 auto;">

se dibuja y parametriza en LTspice:

<img src="./img/GIC_bicuad_LTspice.png" alt="GIC Schaumann" width="800px" style="display: block; margin: 0 auto;">

y luego este archivo será lo que se toma como entrada a las funciones de análisis nodal modificado.


In [1]:
"""
Referencias:
------------
Cap. 5. Schaumann Rolf. Design of Analog Filters.
Cap. 17. Vlach, Jiri - Linear Circuit Theory Matrices in Computer Applications (2014)

"""

import sympy as sp
from pytc2.sistemas_lineales import parametrize_sos
from pytc2.general import s, print_latex, print_subtitle, a_equal_b_latex_s

from pytc2.cuadripolos import smna
from IPython.display import display, Markdown

fileName_asc = './schematics/GIC bicuad.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = False)

print_subtitle('Ecuación MNA')

display(equ_smna)

Utilizando netlist: ./schematics/GIC bicuad.net


#### Ecuación MNA

Eq(Matrix([
[v_va*(a*c0*s + c0*s*(1 - a) + 1/r + (b + eps)/(q*r) + (-b + eps + 1)/(q*r)) + v_vi*(-a*c0*s - (b + eps)/(q*r)) - v_vb/r],
[             I_V1 + v_va*(-a*c0*s - (b + eps)/(q*r)) + v_vi*(a*c0*s + (c + eps)/r + (b + eps)/(q*r)) - v_vd*(c + eps)/r],
[                                                                        I_OXu2 - c0*s*v_vc + v_vb*(c0*s + 1/r) - v_va/r],
[                                                                                -c0*s*v_vb + v_vc*(c0*s + 1/r) - v_vo/r],
[                                                                                    I_OXu1 - v_vc/r - v_vd/r + 2*v_vo/r],
[                                                v_vd*((c + eps)/r + (-c + eps + 1)/r + 1/r) - v_vi*(c + eps)/r - v_vo/r],
[                                                                                                                   v_vi],
[                                                                                                 v_va - v_vc - v_vo/aop],
[   

In [2]:
# tuning a mano de las ecuaciones
A0 = extra_results['A']

if extra_results['eps'] != 0:
    A0 = A0.subs(extra_results['eps'], 0)

if extra_results['aop'] != 0:
    A0 = A0.limit(extra_results['aop'], sp.oo)

equ_smna = sp.Eq(A0*extra_results['X'], extra_results['Z'])

print_subtitle('Ecuación MNA')

display(equ_smna)

#### Ecuación MNA

Eq(Matrix([
[v_va*(a*c0*s + b/(q*r) + c0*s*(1 - a) + 1/r + (1 - b)/(q*r)) + v_vi*(-a*c0*s - b/(q*r)) - v_vb/r],
[                      I_V1 - c*v_vd/r + v_va*(-a*c0*s - b/(q*r)) + v_vi*(a*c0*s + b/(q*r) + c/r)],
[                                                 I_OXu2 - c0*s*v_vc + v_vb*(c0*s + 1/r) - v_va/r],
[                                                         -c0*s*v_vb + v_vc*(c0*s + 1/r) - v_vo/r],
[                                                             I_OXu1 - v_vc/r - v_vd/r + 2*v_vo/r],
[                                               -c*v_vi/r + v_vd*(c/r + (1 - c)/r + 1/r) - v_vo/r],
[                                                                                            v_vi],
[                                                                                     v_va - v_vc],
[                                                                                    -v_vc + v_vd]]), Matrix([
[ 0],
[ 0],
[ 0],
[ 0],
[ 0],
[ 0],
[V1],
[ 0],
[ 0]]))

In [3]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = u1[extra_results['v_out']] / u1[extra_results['v_in']]

display(H)


(2*V1*a*c0**2*q*r**2*s**2 + 2*V1*b*c0*r*s - V1*c*c0**2*q*r**2*s**2 - V1*c*c0*r*s + V1*c*q)/(V1*(c0**2*q*r**2*s**2 + c0*r*s + q))

![nada](./img/nada.png)
<img src="./img/homero1.png" alt="Homero Simpson" align="right" width=100px>


In [4]:
H0 = sp.collect(sp.simplify(sp.expand(H)),s)

H0 = parametrize_sos(H)[0]

display(H0)

((c/(2*a*c0**2*r**2 - c*c0**2*r**2) + s**2 + s*(2*b/(2*a*c0*q*r - c*c0*q*r) - c/(2*a*c0*q*r - c*c0*q*r)))/(s**2 + s/(c0*q*r) + 1/(c0**2*r**2)))*(2*a - c)

![nada](./img/nada.png)
<img src="./img/homero5.png" alt="Homero Simpson" align="right" width=150px>

### Ackerberg Mossberg bicuad

Repetimos el procedimiento, a partir de una red de referencia:

<img src="./img/ACKMOSS_bicuad.png" alt="ACKMOSS" width="800px" style="display: block; margin: 0 auto;">
<img src="./img/ACKMOSS_bicuad2.png" alt="ACKMOSS" width="800px" style="display: block; margin: 0 auto;">

se dibuja y parametriza en LTspice:

<img src="./img/ACKMOSS_bicuad_LTspice.png" alt="ACKMOSS" width="800px" style="display: block; margin: 0 auto;">

y luego procedemos al análisis


In [5]:
fileName_asc = './schematics/ACKMOSS bicuad.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = False)

# tuning a mano de las ecuaciones
A0 = extra_results['A']

if extra_results['eps'] != 0:
    A0 = A0.subs(extra_results['eps'], 0)

if extra_results['aop'] != 0:
    A0 = A0.limit(extra_results['aop'], sp.oo)

equ_smna = sp.Eq(A0*extra_results['X'], extra_results['Z'])

print_subtitle('Ecuación MNA')

display(equ_smna)


Utilizando netlist: ./schematics/ACKMOSS bicuad.net


#### Ecuación MNA

Eq(Matrix([
[v_va*(a*c0*s + c0*s + d/r + 1/r + 1/(q*r)) + v_vi*(-a*c0*s - d/r) + v_vo*(-c0*s - 1/(q*r)) - v_vd/r],
[                I_V2 - b*v_vc/r - c*v_vb/r + v_va*(-a*c0*s - d/r) + v_vi*(a*c0*s + b/r + c/r + d/r)],
[                             I_OXu1 + v_va*(-c0*s - 1/(q*r)) + v_vo*(c0*s + 1/r + 1/(q*r)) - v_vb/r],
[                                                                I_OXu2 - v_va/r - v_vc/r + 2*v_vd/r],
[                                         -c*v_vi/r - c0*s*v_n001 + v_vb*(c/r + c0*s + 1/r) - v_vo/r],
[                                                   -b*v_vi/r + v_vc*(b/r + 2/r) - v_n001/r - v_vd/r],
[                                                  I_OXu3 - c0*s*v_vb + v_n001*(c0*s + 1/r) - v_vc/r],
[                                                                                               v_vi],
[                                                                                              -v_va],
[                                                            

In [6]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = u1[extra_results['v_out']] / u1[extra_results['v_in']]

display(H)

(-V2*a*c0**2*q*r**2*s**2 + V2*b*c0*q*r*s - V2*c*q - V2*c0*d*q*r*s)/(V2*(c0**2*q*r**2*s**2 + c0*r*s + q))

![nada](./img/nada.png)
<img src="./img/homero1.png" alt="Homero Simpson" align="right" width=100px>
 

In [7]:
H0 = sp.collect(sp.simplify(sp.expand(H)),s)

H0 = parametrize_sos(H)[0]

display(H0)

(-a)*((s**2 + c/(a*c0**2*r**2) + s*(-b + d)/(a*c0*r))/(s**2 + s/(c0*q*r) + 1/(c0**2*r**2)))

![nada](./img/nada.png)
<img src="./img/homero5.png" alt="Homero Simpson" align="right" width=150px>
 

### Lattice como ecualizador de fase de primer orden

Analizamos ahora una red pasiva balanceada de primer orden:

<img src="./img/lattice_1ord_LTspice.png" alt="lattice" width="800px" style="display: block; margin: 0 auto;">


In [8]:
fileName_asc = './schematics/lattice_1ord_delay_eq.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = True)

print_subtitle('Ecuación MNA')

display(equ_smna)

Utilizando netlist: ./schematics/lattice_1ord_delay_eq.net


#### Ecuación MNA

Eq(Matrix([
[                I_V1 + v_n001 - v_v1],
[I_L1 - s*v_4 - v_n001 + v_v1*(s + 1)],
[   I_L2 - s*v_v1 - v_3 + v_4*(s + 1)],
[           -I_L1 + v_3*(s + 1) - v_4],
[                              v_n001],
[                -I_L1*s - v_3 + v_v1],
[                       -I_L2*s + v_4]]), Matrix([
[ 0],
[ 0],
[ 0],
[ 0],
[V1],
[ 0],
[ 0]]))

Notar que al tratarse de una salida desbalanceada debemos calcular:
$$ H = \frac{v_3-v_4}{v1-v0} $$

y qye $v_0 = 0$


In [9]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = (u1[extra_results['X'][2]] - u1[extra_results['X'][3]]) / u1[extra_results['X'][1]]

H = sp.simplify(sp.expand(H))

display(H)

(s - 1)/(s + 1)

![nada](./img/nada.png)
<img src="./img/homero5.png" alt="Homero Simpson" align="right" width=150px>
 

### Tee con trafo como ecualizador de fase de primer orden

Analizamos ahora la versión desbalanceada del lattice anteriormente vista:

<img src="./img/tee_puenteada_1ord_LTspice.png" alt="lattice" width="800px" style="display: block; margin: 0 auto;">


In [10]:
fileName_asc = './schematics/tee_1ord_delay_eq.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = True)
print_subtitle('Ecuación MNA')

display(equ_smna)

Utilizando netlist: ./schematics/tee_1ord_delay_eq.net


#### Ecuación MNA

Eq(Matrix([
[                I_V1 + v_n001 - v_vi],
[               -I_L1 - v_n001 + v_vi],
[                         I_L2 + v_vo],
[            I_L1 - I_L2 + 2*s*v_n002],
[                              v_n001],
[-I_L1*s/2 - I_L2*s/2 + v_n002 - v_vi],
[-I_L1*s/2 - I_L2*s/2 - v_n002 + v_vo]]), Matrix([
[ 0],
[ 0],
[ 0],
[ 0],
[V1],
[ 0],
[ 0]]))

In [11]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = u1[extra_results['v_out']] / u1[extra_results['v_in']]

H = sp.simplify(sp.expand(H))

display(H)

(1 - s)/(s + 1)

![nada](./img/nada.png)
<img src="./img/homero5.png" alt="Homero Simpson" align="right" width=150px>
 

### Lattice como ecualizador de fase de segundo orden

Analizamos ahora una red pasiva balanceada de segundo orden:

<img src="./img/lattice_2ord_LTspice.png" alt="lattice2" width="800px" style="display: block; margin: 0 auto;">


In [12]:
fileName_asc = './schematics/lattice_2ord_delay_eq.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = True)

print_subtitle('Ecuación MNA')

display(equ_smna)

Utilizando netlist: ./schematics/lattice_2ord_delay_eq.net


#### Ecuación MNA

Eq(Matrix([
[                                         I_V1 - v_1 + v_n001],
[              I_L1 + I_L2 + v_1*(1 + s/a) - v_n001 - s*v_3/a],
[          -I_L3 - a*s*v_n002/b - v_3 + v_4*(a*s/b + 1 + s/a)],
[-I_L1 - a*s*v_n003/b + v_3*(a*s/b + 1 + s/a) - v_4 - s*v_1/a],
[                            -I_L2 - a*s*v_4/b + a*s*v_n002/b],
[                            -I_L4 - a*s*v_3/b + a*s*v_n003/b],
[                                                      v_n001],
[                                     -I_L1*a*s/b + v_1 - v_3],
[                                    -I_L2*s/a + v_1 - v_n002],
[                                           -I_L3*a*s/b - v_4],
[                                          -I_L4*s/a - v_n003]]), Matrix([
[ 0],
[ 0],
[ 0],
[ 0],
[ 0],
[ 0],
[V1],
[ 0],
[ 0],
[ 0],
[ 0]]))

In [13]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = (u1[extra_results['X'][2]] - u1[extra_results['X'][3]]) / u1[extra_results['X'][1]]

H = sp.simplify(sp.expand(H))

display(H)

(a*s - b - s**2)/(a*s + b + s**2)

![nada](./img/nada.png)
<img src="./img/homero5.png" alt="Homero Simpson" align="right" width=150px>
 

### Tee puenteada como ecualizador de demora de segundo orden

Analizamos ahora una red desbalanceada equivalente sin transformador:

<img src="./img/tee_puent_2do_ord_capa_LTspice.png" alt="tee puent" width="800px" style="display: block; margin: 0 auto;">


In [14]:
fileName_asc = './schematics/tee_puen_2ord_delay_eq.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = True)

# tuning a mano de las ecuaciones
A0 = extra_results['A']

if extra_results['eps'] != 0:
    A0 = A0.subs(extra_results['eps'], 0)

if extra_results['aop'] != 0:
    A0 = A0.limit(extra_results['aop'], sp.oo)

equ_smna = sp.Eq(A0*extra_results['X'], extra_results['Z'])

print_subtitle('Ecuación MNA')

display(equ_smna)


Utilizando netlist: ./schematics/tee_puen_2ord_delay_eq.net


#### Ecuación MNA

Eq(Matrix([
[                     I_V1 + v_n001 - v_vi],
[I_L1 - v_n001 + v_vi*(1 + s/a) - s*v_va/a],
[        -I_L1 + v_vo*(1 + s/a) - s*v_va/a],
[      -I_L2 + 2*s*v_n002/(b*(-a/b + 1/a))],
[  I_L2 + 2*s*v_va/a - s*v_vi/a - s*v_vo/a],
[                                   v_n001],
[              -2*I_L1*a*s/b + v_vi - v_vo],
[            -I_L2*s/(2*a) - v_n002 + v_va]]), Matrix([
[ 0],
[ 0],
[ 0],
[ 0],
[ 0],
[V1],
[ 0],
[ 0]]))

In [15]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = u1[extra_results['v_out']] / u1[extra_results['v_in']]

H0 = sp.collect(sp.simplify(sp.expand(H)),s)

H0 = parametrize_sos(H)[0]

display(H0)

1*((-a*s + b + s**2)/(a*s + b + s**2))

![nada](./img/nada.png)
<img src="./img/homero2.png" alt="Homero Simpson" align="right" width=100px>
 

### Tee puenteada con trafo

Ahora la misma red pero con un transformador para elevar un poco la complejidad:

<img src="./img/tee_puent_2do_ord_trafo_LTspice.png" alt="tee puent trafo" width="800px" style="display: block; margin: 0 auto;">


In [16]:
fileName_asc = './schematics/tee_puen_2ord_delay_eq2.asc'

# symbolic MNA
equ_smna, extra_results = smna(fileName_asc, 
                               bAplicarValoresComponentes = True, 
                               bAplicarParametros = True)

# tuning a mano de las ecuaciones
A0 = extra_results['A']

if extra_results['eps'] != 0:
    A0 = A0.subs(extra_results['eps'], 0)

if extra_results['aop'] != 0:
    A0 = A0.limit(extra_results['aop'], sp.oo)

equ_smna = sp.Eq(A0*extra_results['X'], extra_results['Z'])

print_subtitle('Ecuación MNA')

display(equ_smna)

Utilizando netlist: ./schematics/tee_puen_2ord_delay_eq2.net


#### Ecuación MNA

Eq(Matrix([
[                                                                                    I_V1 + v_n001 - v_vi],
[                                                      -I_L1 - v_n001 + v_vi*(1 + s/(2*a)) - s*v_vo/(2*a)],
[                                                               -I_L2 + v_vo*(1 + s/(2*a)) - s*v_vi/(2*a)],
[                                                                              I_L1 + I_L2 + 2*a*s*v_va/b],
[                                                                                                  v_n001],
[-I_L1*s*(a/(2*b) + 1/(2*a)) - I_L2*s*(-a/b + 1/a)*sqrt((a/(2*b) + 1/(2*a))**2)/(a/b + 1/a) + v_va - v_vi],
[-I_L1*s*(-a/b + 1/a)*sqrt((a/(2*b) + 1/(2*a))**2)/(a/b + 1/a) - I_L2*s*(a/(2*b) + 1/(2*a)) + v_va - v_vo]]), Matrix([
[ 0],
[ 0],
[ 0],
[ 0],
[V1],
[ 0],
[ 0]]))

![nada](./img/nada.png)
<img src="./img/homero3.png" alt="Homero Simpson" align="right" width=100px>
 

In [17]:
u1 = sp.solve(equ_smna, extra_results['X'])

H = u1[extra_results['v_out']] / u1[extra_results['v_in']]

display(H)

(-2*V1*a**5*b*s**2*sqrt(a**2/b**2 + 2/b + a**(-2))/(4*a**6*s**2 + 6*a**5*b*s + 6*a**5*s**3 + 2*a**4*b**2*s*sqrt(a**2/b**2 + 2/b + a**(-2)) + 4*a**4*b**2 + 2*a**4*b*s**3*sqrt(a**2/b**2 + 2/b + a**(-2)) + 10*a**4*b*s**2 + 4*a**4*s**4 + 2*a**3*b**2*s**2*sqrt(a**2/b**2 + 2/b + a**(-2)) + 8*a**3*b**2*s + 8*a**3*b*s**3 - 2*a**2*b**3*s*sqrt(a**2/b**2 + 2/b + a**(-2)) + 4*a**2*b**3 - 2*a**2*b**2*s**3*sqrt(a**2/b**2 + 2/b + a**(-2)) + 8*a**2*b**2*s**2 + 4*a**2*b*s**4 - 2*a*b**3*s**2*sqrt(a**2/b**2 + 2/b + a**(-2)) + 2*a*b**3*s + 2*a*b**2*s**3 + 2*b**3*s**2) + 2*V1*a**4*b**2/(4*a**6*s**2 + 6*a**5*b*s + 6*a**5*s**3 + 2*a**4*b**2*s*sqrt(a**2/b**2 + 2/b + a**(-2)) + 4*a**4*b**2 + 2*a**4*b*s**3*sqrt(a**2/b**2 + 2/b + a**(-2)) + 10*a**4*b*s**2 + 4*a**4*s**4 + 2*a**3*b**2*s**2*sqrt(a**2/b**2 + 2/b + a**(-2)) + 8*a**3*b**2*s + 8*a**3*b*s**3 - 2*a**2*b**3*s*sqrt(a**2/b**2 + 2/b + a**(-2)) + 4*a**2*b**3 - 2*a**2*b**2*s**3*sqrt(a**2/b**2 + 2/b + a**(-2)) + 8*a**2*b**2*s**2 + 4*a**2*b*s**4 - 2*a*b**3*s**2*

![nada](./img/nada.png)
<img src="./img/homero6.png" alt="Homero Simpson" align="right" width=150px>
 

In [18]:
H0 = H.subs(extra_results['dic_params'])

H0 = sp.collect(sp.simplify(sp.expand(H0)),s)

#H0 = parametrize_sos(H)[0]

display(H0)

(s**4 + 1)*(s**4 + 2*sqrt(2)*s**3 + 4*s**2 + 2*sqrt(2)*s + 1)/(s**8 + 4*sqrt(2)*s**7 + 16*s**6 + 20*sqrt(2)*s**5 + 34*s**4 + 20*sqrt(2)*s**3 + 16*s**2 + 4*sqrt(2)*s + 1)

![nada](./img/nada.png)
<img src="./img/homero4.png" alt="Homero Simpson" align="right" width=150px>
 

No es el resultado que esperábamos, habrá que ver dónde quedó un error ...